In [17]:
import pandas as pd

In [18]:
ml_movies = pd.read_csv('../raw_data/ml-latest-small/movies.csv') # has titles
ml_movie_ratings = pd.read_csv('../raw_data/ml-latest-small/ratings.csv')

In [19]:
ml_movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [20]:
ml_movie_ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [21]:
ml_movie_titles_ratings = ml_movies.merge(ml_movie_ratings, on="movieId", how="right")[['title', 'userId', 'rating', 'movieId', 'genres']]

In [22]:
# handle movie reboots -- i.e., same name but diff year + ratings
# just keep the first one, even if they don't all correspond to descriptions from netflix dataset
# noise is good for the heart
ml_movie_titles_ratings['year'] = ml_movie_titles_ratings['title'].str.extract(r" \((\d+)\)")
ml_movie_titles_ratings['title'] = \
  ml_movie_titles_ratings['title'].str.replace(r' \([0-9]{4}\)$', '')
ml_movie_titles_ratings

/var/folders/3q/46mhvg_95v3dkwjv0vvp3r1r0000gn/T/ipykernel_54618/4091935637.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  ml_movie_titles_ratings['title'].str.replace(r' \([0-9]{4}\)$', '')


,title,userId,rating,movieId,genres,year
0,Toy Story,1,4.0,1,Adventure|Animation|Children|Comedy|Fantasy,1995
1,Grumpier Old Men,1,4.0,3,Comedy|Romance,1995
2,Heat,1,4.0,6,Action|Crime|Thriller,1995
3,Seven (a.k.a. Se7en),1,5.0,47,Mystery|Thriller,1995
4,"Usual Suspects, The",1,5.0,50,Crime|Mystery|Thriller,1995
...,...,...,...,...,...,...
100831,Split,610,4.0,166534,Drama|Horror|Thriller,2017
100832,John Wick: Chapter Two,610,5.0,168248,Action|Crime|Thriller,2017
100833,Get Out,610,5.0,168250,Horror,2017
100834,Logan,610,5.0,168252,Action|Sci-Fi,2017


In [23]:
ml_movie_titles_ratings[ml_movie_titles_ratings['title'] == 'Jungle Book, The']

,title,userId,rating,movieId,genres,year
21,"Jungle Book, The",1,5.0,362,Adventure|Children|Romance,1994
127,"Jungle Book, The",1,5.0,2078,Animation|Children|Comedy|Musical,1967
425,"Jungle Book, The",4,5.0,2078,Animation|Children|Comedy|Musical,1967
731,"Jungle Book, The",6,3.0,362,Adventure|Children|Romance,1994
2341,"Jungle Book, The",19,2.0,362,Adventure|Children|Romance,1994
...,...,...,...,...,...,...
93300,"Jungle Book, The",599,3.0,2078,Animation|Children|Comedy|Musical,1967
95172,"Jungle Book, The",600,2.5,362,Adventure|Children|Romance,1994
95402,"Jungle Book, The",600,3.5,2078,Animation|Children|Comedy|Musical,1967
96044,"Jungle Book, The",602,3.0,362,Adventure|Children|Romance,1994


In [24]:
first_of_title_and_year = ml_movie_titles_ratings.drop_duplicates('title')

In [25]:
first_of_title_and_year[first_of_title_and_year['title'] == 'Jungle Book, The']

,title,userId,rating,movieId,genres,year
21,"Jungle Book, The",1,5.0,362,Adventure|Children|Romance,1994


In [26]:
unique_movies = ml_movie_titles_ratings.drop_duplicates('title', keep=False)

In [27]:
unique_movies = pd.concat([first_of_title_and_year, unique_movies], ignore_index=True)

In [28]:
unique_movies[unique_movies['title'] == 'Jungle Book, The']

,title,userId,rating,movieId,genres,year
21,"Jungle Book, The",1,5.0,362,Adventure|Children|Romance,1994


In [29]:
keys = ['title', 'year']
i1 = ml_movie_titles_ratings.set_index(keys).index
i2 = unique_movies.set_index(keys).index
ml_movie_titles_ratings = ml_movie_titles_ratings[i1.isin(i2)]

In [30]:
ml_movie_titles_ratings[ml_movie_titles_ratings['title'] == 'Jungle Book, The']

,title,userId,rating,movieId,genres,year
21,"Jungle Book, The",1,5.0,362,Adventure|Children|Romance,1994
731,"Jungle Book, The",6,3.0,362,Adventure|Children|Romance,1994
2341,"Jungle Book, The",19,2.0,362,Adventure|Children|Romance,1994
2987,"Jungle Book, The",20,4.0,362,Adventure|Children|Romance,1994
5627,"Jungle Book, The",40,4.0,362,Adventure|Children|Romance,1994
6375,"Jungle Book, The",43,5.0,362,Adventure|Children|Romance,1994
6520,"Jungle Book, The",45,5.0,362,Adventure|Children|Romance,1994
8577,"Jungle Book, The",58,5.0,362,Adventure|Children|Romance,1994
8734,"Jungle Book, The",60,4.0,362,Adventure|Children|Romance,1994
14742,"Jungle Book, The",93,4.0,362,Adventure|Children|Romance,1994


In [31]:
ml_movie_titles_ratings.drop('year', axis=1, inplace=True)

/var/folders/3q/46mhvg_95v3dkwjv0vvp3r1r0000gn/T/ipykernel_54618/3485331211.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ml_movie_titles_ratings.drop('year', axis=1, inplace=True)


In [32]:
netflix_descrips = pd.read_csv('../raw_data/netflix_titles.csv')[['title', 'description']]

In [33]:
ml_netflix = netflix_descrips.merge(ml_movie_titles_ratings, on="title", how="inner")

In [34]:
ml_netflix[ml_netflix['title'] == 'Grown Ups']

,title,description,userId,rating,movieId,genres
0,Grown Ups,Mourning the loss of their beloved junior high...,21,2.0,79134,Comedy
1,Grown Ups,Mourning the loss of their beloved junior high...,103,3.5,79134,Comedy
2,Grown Ups,Mourning the loss of their beloved junior high...,111,5.0,79134,Comedy
3,Grown Ups,Mourning the loss of their beloved junior high...,249,3.0,79134,Comedy
4,Grown Ups,Mourning the loss of their beloved junior high...,298,2.5,79134,Comedy
5,Grown Ups,Mourning the loss of their beloved junior high...,318,4.0,79134,Comedy
6,Grown Ups,Mourning the loss of their beloved junior high...,341,4.5,79134,Comedy
7,Grown Ups,Mourning the loss of their beloved junior high...,424,3.0,79134,Comedy
8,Grown Ups,Mourning the loss of their beloved junior high...,448,3.5,79134,Comedy
9,Grown Ups,Mourning the loss of their beloved junior high...,534,3.5,79134,Comedy


In [35]:
from utils import reassign_ids
ml_netflix = reassign_ids(ml_netflix, 'movieId')
ml_netflix = reassign_ids(ml_netflix, 'userId')

In [36]:
ml_netflix.to_csv('../ml_netflix.csv', index=False)